In [ ]:
# Load library imports
import os
import sys
import torch
import random
import logging
import numpy as np
import pandas as pd
import geopandas as gpd

# Load project Imports
from src.data_ingestion import static_data_ingestion
from src.utils.config_loader import load_project_config
from src.graph_building.graph_construction import build_mesh, \
    define_catchment_polygon, build_main_df
from src.graph_building.data_merging import snap_stations_to_mesh, \
    merge_timeseries_data_to_df, load_gwl_data_for_merge
from src.visualisation.mapped_visualisations import plot_interactive_mesh_with_stations
from src.preprocessing.hydroclimatic_feature_engineering import transform_aet_data

In [ ]:
# Set up logger config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/project_config.yaml")
notebook = True

# Set up seeding to define global states
random_seed = config["global"]["pipeline_settings"]["random_seed"]
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define notebook demo catchment
catchments_to_process = config["global"]["pipeline_settings"]["catchments_to_process"]
catchment = catchments_to_process[0]
run_defra_API_calls = config["global"]["pipeline_settings"]["run_defra_api"]

logger.info(f"Show Notebook Outputs: {notebook}")
logger.info(f"Notebook Demo Catchment: {catchment.capitalize()}")

In [ ]:
# Select Catchment area from country wide gdf
define_catchment_polygon(
    england_catchment_gdf_path=config[catchment]['paths']['gis_catchment_boundary'],
    target_mncat=config[catchment]['target_mncat'],
    catchment=catchment,
    polygon_output_path=config[catchment]['paths']['gis_catchment_dir']
)

# Build catchment mesh
mesh_nodes_table, mesh_nodes_gdf, mesh_cells_gdf_polygons, catchment_polygon = build_mesh(
    shape_filepath=config[catchment]['paths']['gis_catchment_dir'],
    output_path=config[catchment]['paths']['mesh_nodes_output'],
    catchment=catchment,
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution']
)

logger.info(f"Pipeline step 'Build Mesh' complete for {catchment} catchment.")

Load in centroid node csv's

In [ ]:
# input_path=config[catchment]['paths']['mesh_nodes_output']
# grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution']

# mesh_input_path = input_path + '_' + str(grid_resolution) + '.csv'
# mesh_nodes = pd.read_csv(mesh_input_path)

Merge station list with polygon geometry using spatial join to snap stations to mesh

In [ ]:
station_node_mapping = snap_stations_to_mesh(
    station_list_path=config[catchment]["paths"]["gwl_station_list_output"],
    polygon_geometry_path=config[catchment]['paths']['output_polygon_dir'],
    output_path=config[catchment]["paths"]["snapped_station_node_mapping"],
    mesh_nodes_gdf=mesh_nodes_gdf,
    catchment=catchment
)

In [ ]:
station_node_mapping

In [ ]:
mesh_map = plot_interactive_mesh_with_stations(
    mesh_nodes_gdf=mesh_nodes_gdf,
    catchment_polygon=catchment_polygon,
    map_blue=config['global']['visualisations']['maps']['map_blue'],
    esri=config['global']['visualisations']['maps']['esri'],
    esri_attr=config['global']['visualisations']['maps']['esri_attr'],
    static_output_path=config[catchment]['visualisations']['maps']['static_mesh_map_output'],
    interactive_output_path=config[catchment]['visualisations']['maps']['interactive_station_map_output'],
    catchment=catchment,
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution'],
    interactive=config['global']['visualisations']['maps']['display_interactive_map'],
    stations_gdf=station_node_mapping
)

logger.info(f"Pipeline step 'Interactive Mesh Mapping' complete for {catchment} catchment.")

# Display map in notebook
mesh_map

Create a main df for merging all features in to model input

In [ ]:
main_df = build_main_df(
    start_date=config["global"]["data_ingestion"]["model_start_date"],
    end_date=config["global"]["data_ingestion"]["model_end_date"],
    mesh_nodes_gdf=mesh_nodes_gdf,
    catchment=catchment
)

Merge all timeseries data into main df by station (from {station}_trimmed.csv)

### Merge in Timeseries Features ###

In [ ]:
# Snap Precipitation, Lags and Averages to timestep

merged_ts_precipitation = merge_timeseries_data_to_df(
    model_start_date=config["global"]["data_ingestion"]["model_start_date"],
    model_end_date=config["global"]["data_ingestion"]["model_end_date"],
    feature_csv=config[catchment]["paths"]["rainfall_csv_path"],
    feature='all_precipitation'
)

logger.info(f"Precipitation and derived data snapped to graph timesteps (daily aggregates).\n")

# Snap 2m Temperature to timestep

merged_ts_tsm = merge_timeseries_data_to_df(
    model_start_date=config["global"]["data_ingestion"]["model_start_date"],
    model_end_date=config["global"]["data_ingestion"]["model_end_date"],
    feature_csv=config[catchment]["paths"]["2t_csv_path"],
    feature='2m_temperature',
    timeseries_df=merged_ts_precipitation
)

logger.info(f"2m Temperature Data snapped to graph timesteps (daily aggregate).\n")

# Snap AET to timestep

merged_ts_aet = merge_timeseries_data_to_df(
    model_start_date=config["global"]["data_ingestion"]["model_start_date"],
    model_end_date=config["global"]["data_ingestion"]["model_end_date"],
    feature_csv=config[catchment]["paths"]["aet_csv_path"],
    feature='aet',
    timeseries_df=merged_ts_tsm
)
        
merged_ts_aet = transform_aet_data(merged_ts_aet, catchment)

logger.info(f"Actual evapotranspiration data snapped to graph timesteps (daily aggregate).\n")

# Snap Surface Pressure to timestep

final_merged_ts_df = merge_timeseries_data_to_df(
    model_start_date=config["global"]["data_ingestion"]["model_start_date"],
    model_end_date=config["global"]["data_ingestion"]["model_end_date"],
    feature_csv=config[catchment]["paths"]["sp_csv_path"],
    feature='surface_pressure',
    timeseries_df=merged_ts_aet
)

logger.info(f"Surface pressure data snapped to graph timesteps (daily aggregate).\n")

save_path = config[catchment]["paths"]["final_df_path"] + 'final_timeseries_df.csv'
final_merged_ts_df.to_csv(save_path)

logger.info(f"Final merged time series dataframe saved to {save_path}\n")

In [ ]:
final_merged_ts_df

### Merge in Static Features ###

In [ ]:
# Snap Land Cover to Mesh

agg_land_cover_df = pd.read_csv(config[catchment]["paths"]["land_cover_csv_path"])
merged_gdf_nodes_landuse = mesh_nodes_gdf.merge(
    agg_land_cover_df[['easting', 'northing', 'land_cover_code']],
    on=['easting', 'northing'],
    how='left'  # left join to keep all centroids, even NaN
)

logger.info(f"Land cover data snapped to mesh nodes (centroids).\n")

# Snap Elevation to Mesh

elevation_gdf_polygon=gpd.read_file(config[catchment]["paths"]["elevation_geojson_path"])
merged_gdf_nodes_elevation = merged_gdf_nodes_landuse.merge(
    elevation_gdf_polygon[['node_id', 'mean_elevation', 'geometry']],
    on='node_id',
    how='left'  # left join to keep all centroids, even NaN
)

logger.info(f"Elevation data snapped to mesh nodes (centroids).\n")

# Snap Slope to Mesh [TODO: THIS IS CURRENTLY ACTING AS FINAL STATIC DF, UPDATE IN FUTURE]

slope_gdf = pd.read_csv(config[catchment]["paths"]["slope_path"])
# merged_gdf_nodes_slope = merged_gdf_nodes_elevation.merge(
static_features = merged_gdf_nodes_elevation.merge(
    slope_gdf[['node_id', 'mean_slope_degrees', 'mean_aspect_sin', 'mean_aspect_cos']],
    on='node_id',
    how='left'  # left join to keep all centroids, even NaN
)

logger.info(f"Slope degrees and sinusoidal aspect data snapped to mesh nodes (centroids).\n")

# Strip out features not required for merge into main_df
static_data_ingestion.save_final_static_data(
    static_features=static_features,
    dir_path=config[catchment]["paths"]["final_df_path"]
)

static_features

### Final Merge into main_df ###

In [ ]:
# Merge static data into main_df
static_df = pd.read_csv(os.path.join(config[catchment]["paths"]["final_df_path"], 'final_static_df.csv'))
main_df_static = main_df.merge(
    static_df,
    left_on='node_id',
    right_on='node_id',
    how='left'
)

logger.info(f"Static data successfully merged into main_df for {catchment} catchment.\n")

# Merge timeseries data into main_df
timeseries_df = pd.read_csv(os.path.join(config[catchment]["paths"]["final_df_path"], 'final_timeseries_df.csv'))
timeseries_df['time'] = pd.to_datetime(timeseries_df['time'])
main_df_timeseries = main_df_static.merge(
    timeseries_df,
    left_on='timestep',
    right_on='time',
    how='left'
).drop(columns='time')

logger.info(f"Timeseries data successfully merged into main_df for {catchment} catchment.\n")
    
# Load GWL station data and merge data into main_df
station_dir = config[catchment]["paths"]["trimmed_output_dir"]
node_mapping_dir = config[catchment]["paths"]["snapped_station_node_mapping"]
gwl_data = load_gwl_data_for_merge(station_dir, node_mapping_dir)
main_df_full = main_df_timeseries.merge(
    gwl_data,
    on=['node_id', 'timestep'],
    how='left'
)

logger.info(f"Groundwater Level data successfully merged into main_df for {catchment} catchment.\n")

# Save final dataframe to file
final_save_path = config[catchment]["paths"]["final_df_path"] + 'final_df.csv'
# main_df_full.to_csv(final_save_path)
logger.info(f"Final merged dataframe saved to {final_save_path}")

# Display in notebook
main_df_full